In [70]:
import os
from glob import glob
from random import shuffle
import numpy as np
import skimage.io as io
import skimage.transform as transform
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cv2
import pandas as pd

from PIL import Image
from skimage.filters import gaussian
from skimage.morphology import erosion
#%matplotlib inline

In [2]:
# def foregroundAug(foreground):
#     # Random rotation, zoom, translation
#     angle = np.random.randint(-10, 10) * (np.pi / 180.0)  # Convert to radians
#     zoom = np.random.random() * 0.4 + 0.8  # Zoom in range [0.8,1.2)
#     zoom = np.random.random() * 0.3 + 0.5  # Zoom in range [0.8,1.2)

#     tform = transform.AffineTransform(scale=(zoom, zoom),
#                                       rotation=angle)
#     #  ,translation=(t_x, t_y))
#     skew = np.random.uniform(low=-1, high=1) * 0.3
#     rows = foreground.shape[0]
#     cols = foreground.shape[1]
#     ratioY = skew * cols / rows
#     skew = np.random.uniform(low=-1, high=1) * 0.3
#     ratioX = skew * cols / rows
# #     matrix = np.array([[1, ratioY, 0], [0, 1, 0], [0, 0, 1]])
#     matrix = np.array([[1, ratioY, 0], [ratioX, 1, 0], [0, 0, 1]])
#     tform2 = transform.ProjectiveTransform(matrix=matrix)

#     foreground = transform.warp(foreground, tform2, cval=-1)  # tform.inverse)
#     #     foreground = transform.warp(foreground, tform.inverse)

#     # Random horizontal flip with 0.5 probability
#     #     if(np.random.randint(0,100)>=50):
#     #         foreground = foreground[:, ::-1]

#     return foreground

In [3]:
def make_folder(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [4]:
def arr_2_pil(image):
    
    img = image.copy()
    img -= img.min()
    img /= img.max()
    img = np.uint8(img*255)
    img = Image.fromarray(img)
    
    return img

In [5]:
def read_png_image_mask(path):
    
    # read png
    image = plt.imread(path)
    # separate the alpha channel
    mask = mask = image[:, :, -1].copy()
    # separate the rgb channels
    image = image[:, :, :-1].copy()
    # reshape mask to have 3 channels
    mask = np.concatenate([np.expand_dims(mask, axis=2) for i in range(3)], axis=2)
    
    return image, mask

In [6]:
def pad_image_mask(image, mask, padding=0.3):
    
    H, W, C = image.shape
    padding = int(padding*max(H, W))
    
    p_H = H + 2*padding
    p_W = W + 2*padding
    
    padded_image = np.zeros((p_H, p_W, 3))
    padded_mask = padded_image.copy()
    
    padded_image[padding:H+padding, padding:W+padding,:] = image
    padded_mask[padding:H+padding, padding:W+padding,:] = mask
    
    return padded_image, padded_mask

In [7]:
def foregroundAug(image, mask, skew_max=0.3):
    
    skew = np.random.uniform(low=-1, high=1) * skew_max
    matrix = np.array([[1, skew, 0], [0, 1, 0], [0, 0, 1]])
    tform = transform.ProjectiveTransform(matrix=matrix)
    
    image_tform = transform.warp(image, tform)
    mask_tform = transform.warp(mask, tform)
    
    return image_tform, mask_tform

In [8]:
def crop_padding(image, mask):
    
    mask_sum = mask.sum(axis=2)
    
    h_results = mask_sum.sum(axis=1)
    hmin, hmax = np.where(h_results!=0)[0][np.array([0, -1])]
    
    w_results = mask_sum.sum(axis=0)
    wmin, wmax = np.where(w_results!=0)[0][np.array([0, -1])]
    
    image_crop, mask_crop = image.copy(), mask.copy()
    
    image_crop = image_crop[hmin:hmax, wmin:wmax]
    mask_crop = mask_crop[hmin:hmax, wmin:wmax]
    
    return image_crop, mask_crop

In [9]:
def add_noise(image, noise_type='random', noise_weight=None, noise_range=(0.1, 0.85)):
    
    image_shape = image.shape
    
    if not noise_weight:
        nmin, nmax = noise_range
        noise_weight = np.random.uniform(low=nmin, high=nmax)
    
    if noise_type == 'random':
        noise = np.random.random(size=image_shape)
        
    elif noise_type == "gauss":
        mean = 0
        var = 0.1
        sigma = var ** 0.5
        noise = np.random.normal(mean, sigma, size=image_shape)
        
        
    return image * (1-noise_weight) + noise * noise_weight

In [10]:
def add_gaussian_blur(image, sigma=None, sigma_range=(1, 21)):
    
    if not sigma:
        smin, smax = sigma_range
        sigma = np.random.randint(smin, smax)
    
    image = gaussian(image, sigma=sigma, multichannel=True, preserve_range=True)
    
    return image

In [11]:
def scale_image_mask(image, mask, dim=32):
    
    H, W, C = image.shape
    
    large_dim = max(H, W)
    
    scale = dim / large_dim
    
    H = int(np.floor(H*scale))
    W = int(np.floor(W*scale))
    
    im = image.copy()
    ms = mask.copy()
    
    im -= im.min()
    im /= im.max()
    
    im = Image.fromarray(np.uint8(im*255))
    ms = Image.fromarray(np.uint8(ms*255))
    
    im = im.resize((W, H))
    ms = ms.resize((W, H))
    
    im = np.array(im) / 255
    ms = np.array(ms) / 255
    
    return im, ms

In [12]:
# from scipy.stats import truncnorm
# mean, sd, low, upp = (56, 53, 20, 650)
# gen = truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)
# sizes = np.uint8(gen.rvs(2000).round())

def generate_sizes(size=10):
    
    most = round(size * 0.85)
    less = size - most
    most = np.random.randint(20, 100, size=most)
    less = np.random.randint(100, 300, size=less)
    
    out = np.concatenate([most, less])
    shuffle(out)
    return out

In [13]:
def get_augmented_image_mask(path, sizes=np.array([16, 32, 64, 128, 256, 512])):
    
    im, mask = read_png_image_mask(path)
    im_pad, mask_pad = pad_image_mask(im, mask, padding=0.5)
    im_aug, mask_aug = foregroundAug(im_pad, mask_pad, skew_max=0.3)
    im_aug, mask_aug = crop_padding(im_aug, mask_aug)
    
    im_aug = add_noise(im_aug, noise_weight=None, noise_type='random', noise_range=(0.5, 0.85))
    im_aug = add_gaussian_blur(im_aug, sigma=None, sigma_range=(5, 16))
    im_aug, mask_aug = scale_image_mask(im_aug, mask_aug, dim=np.random.choice(sizes))
    
    im_aug = add_noise(im_aug, noise_weight=None, noise_type='random', noise_range=(0, 0.1))
    mask_aug[mask_aug<1]=0
    mask_aug = erosion(mask_aug)
    
    return im_aug, mask_aug

In [14]:
background_df = pd.read_csv('./no_sign_images.csv')
background_path = '../../blyncsy_signs/tdot/images/'
background_source = np.array([os.path.join(background_path, fl) for fl in background_df['filename']])

sign_source = glob('./sign_source/*.png')
sizes = generate_sizes(size=1000)

In [15]:
# background_seq=[[0, 1, 2, 5, 6, 7, 8],
# [0, 1, 3, 5, 6, 7, 8],
# [0, 1, 4, 5, 6, 7, 8]]
# Image.open(background_source[8])
import time

In [48]:
sign_back_path=glob('../images_mike/back/*.jpg')

In [49]:
len(sign_back_path)

122

In [52]:
sign_back_outPath = './synth_classification/back'
if not os.path.isdir(sign_back_outPath):
    os.mkdir(sign_back_outPath)

In [69]:
%%time
for fl in sign_back_path:
    im = Image.open(fl)
    im_flip = im.transpose(Image.FLIP_LEFT_RIGHT)
    out_name = f"{os.path.basename(fl)[:-4]}_flip.jpg"
    im_flip.save(os.path.join(sign_back_outPath, out_name))

CPU times: user 29.9 ms, sys: 32.4 ms, total: 62.3 ms
Wall time: 327 ms


In [36]:
%%time
start_time=time.time()
count=0

for background_path in background_source[np.array([0, 1, 2, 5, 6, 7, 8])]:
    
    background_original = Image.open(background_path)
    background_original = np.array(background_original) / 255.0
    YY, XX, _ = background_original.shape
    
    shuffle(sign_source)
    
    idx0, idx1 = 0, 0
    idxMax = len(sign_source)-1
    while idx1 < idxMax:
        
        idx1 = np.random.randint(low=1, high=5) + idx0
        idx1 = idxMax if idx1 > idxMax else idx1
        
        paths = sign_source[idx0:idx1]
        
        idx0 = idx1
        
        

        # read background
        background = background_original.copy()

        # get # of signs to place
        num_images = len(paths)

        synth_images=[]
        max_w=0
        max_h=0

        for p in paths:
            # get augmented sign and mask
            im, mask = get_augmented_image_mask(p, sizes=sizes)
            # record max width and max height
            H, W, _ = im.shape
            if W > max_w:
                max_w=W
            if H > max_h:
                max_h=H

            # get folder name
            folder_name = p.split('_')[-1][:-4]
            # make sample dictionary
            synth_images.append({'image': im, 'mask': mask, 'shape': (H, W), 'folder': folder_name})

        # determine offset (divide image into sections)
        offset = (XX)//num_images

        for i in range(num_images):
            # create ranges where the image can be placed
            synth_images[i]['range'] = (i*offset, i*offset+offset-max_w, YY-max_h)

        for i, sample in enumerate(synth_images):

            xmin, xmax, ymax = sample['range']
            im, mask = sample['image'], sample['mask']
            yy, xx = sample['shape']

            border=80
            py = np.random.randint(0, ymax-border)+border
            px = np.random.randint(xmin, xmax)

            sign_overlay = background[py:py+yy, px:px+xx].copy()
            im[mask < 1] = sign_overlay[mask < 1]

            gamma = np.random.uniform(low=-0.1, high=0.1)
            im = cv2.addWeighted(sign_overlay, 0.12, im, 0.8, gamma)
            im[mask < 1] = sign_overlay[mask < 1]
            im[im<0]=0
            im[im>1]=1

            synth_images[i]['image']=im
            background[py:py+yy, px:px+xx]=im
            
            
            # write annotation information
            img_file_name = f"synth_{count:04}.jpg"
            Folder = synth_images[i]['folder']
            
            annot = f"{img_file_name}\t{Folder}\t{px},{py},{xx},{yy}\t{px}\t{py}\t{xx}\t{yy}\t{xx * yy}\n"
            if os.path.isfile("Detection_labels_Syn.txt"):
                with open("Detection_labels_Syn.txt", "a") as fl:
                    fl.write(annot)
            else:
                with open("Detection_labels_Syn.txt", "a") as fl:
                    fl.write("filename\tclass\tbbox\tx0\ty0\tw\th\tarea\n")
                    fl.write(annot)
                    
            # write classification images
            fldr_class_out = os.path.join('./synth_classification', Folder)
            make_folder(fldr_class_out)
            im_out = arr_2_pil(im)
            im_out.save(os.path.join(fldr_class_out, f"{count:04}_{i:04}.jpg"))

        
        fldr_detc_out = os.path.join('./synth_detection', img_file_name)
        im_out = arr_2_pil(background)
        im_out.save(fldr_detc_out)
        
        
        if count % 50 == 0:
            time_elapsed = time.time() - start_time
            print(f"Count: {count}, Time: {round(time_elapsed/3600, 3)}")
        
        count+=1

Count: 0, Time: 0.0
Count: 50, Time: 0.028
Count: 100, Time: 0.054
Count: 150, Time: 0.079
Count: 200, Time: 0.106
Count: 250, Time: 0.132
Count: 300, Time: 0.158
Count: 350, Time: 0.185
Count: 400, Time: 0.208
Count: 450, Time: 0.236
Count: 500, Time: 0.262
Count: 550, Time: 0.288
Count: 600, Time: 0.317
Count: 650, Time: 0.343
Count: 700, Time: 0.369
Count: 750, Time: 0.397
Count: 800, Time: 0.425
Count: 850, Time: 0.451
Count: 900, Time: 0.48
Count: 950, Time: 0.51
Count: 1000, Time: 0.534
Count: 1050, Time: 0.559
Count: 1100, Time: 0.586
Count: 1150, Time: 0.612
Count: 1200, Time: 0.637
Count: 1250, Time: 0.66
Count: 1300, Time: 0.686
Count: 1350, Time: 0.716
Count: 1400, Time: 0.744
Count: 1450, Time: 0.771
Count: 1500, Time: 0.798
Count: 1550, Time: 0.823
Count: 1600, Time: 0.845
Count: 1650, Time: 0.874
Count: 1700, Time: 0.902
Count: 1750, Time: 0.926
CPU times: user 46min 47s, sys: 7min 10s, total: 53min 58s
Wall time: 56min 14s


In [ ]:
%%time
start_time=time.time()
count=0

num_class_img=100

for k in range(num_class_img):
    
    mod = k%3

    if mod == 0:
        back_select = np.array([0, 1, 2, 5, 6, 7, 8])
    elif mod == 1:
        back_select = np.array([0, 1, 3, 5, 6, 7, 8])
    elif mod == 2:
        pback_select = np.array([0, 1, 4, 5, 6, 7, 8])
    

    for background_path in background_source[back_select]:

        background_original = Image.open(background_path)
        background_original = np.array(background_original) / 255.0
        YY, XX, _ = background_original.shape

        shuffle(sign_source)

        idx0, idx1 = 0, 0
        idxMax = len(sign_source)-1
        while idx1 < idxMax:

    #         idx1 = np.random.randint(low=1, high=5) + idx0
            idx1 = 4 + idx0
            idx1 = idxMax if idx1 > idxMax else idx1

            paths = sign_source[idx0:idx1]

            idx0 = idx1



            # read background
            background = background_original.copy()

            # get # of signs to place
            num_images = len(paths)

            synth_images=[]
            max_w=0
            max_h=0

            for p in paths:
                # get augmented sign and mask
                im, mask = get_augmented_image_mask(p, sizes=sizes)
                # record max width and max height
                H, W, _ = im.shape
                if W > max_w:
                    max_w=W
                if H > max_h:
                    max_h=H

                # get folder name
                folder_name = p.split('_')[-1][:-4]
                # make sample dictionary
                synth_images.append({'image': im, 'mask': mask, 'shape': (H, W), 'folder': folder_name})

            # determine offset (divide image into sections)
            offset = (XX)//num_images

            for i in range(num_images):
                # create ranges where the image can be placed
                synth_images[i]['range'] = (i*offset, i*offset+offset-max_w, YY-max_h)

            for i, sample in enumerate(synth_images):

                xmin, xmax, ymax = sample['range']
                im, mask = sample['image'], sample['mask']
                yy, xx = sample['shape']

                border=80
                py = np.random.randint(0, ymax-border)+border
                px = np.random.randint(xmin, xmax)

                sign_overlay = background[py:py+yy, px:px+xx].copy()
                im[mask < 1] = sign_overlay[mask < 1]

                gamma = np.random.uniform(low=-0.1, high=0.1)
                im = cv2.addWeighted(sign_overlay, 0.12, im, 0.8, gamma)
                im[mask < 1] = sign_overlay[mask < 1]
                im[im<0]=0
                im[im>1]=1

                synth_images[i]['image']=im
                background[py:py+yy, px:px+xx]=im


                # write annotation information
#                 img_file_name = f"synthClass_{count:04}.jpg"
                Folder = synth_images[i]['folder']

    #             annot = f"{img_file_name}\t{Folder}\t{px},{py},{xx},{yy}\t{px}\t{py}\t{xx}\t{yy}\t{xx * yy}\n"
    #             if os.path.isfile("Detection_labels_Syn.txt"):
    #                 with open("Detection_labels_Syn.txt", "a") as fl:
    #                     fl.write(annot)
    #             else:
    #                 with open("Detection_labels_Syn.txt", "a") as fl:
    #                     fl.write("filename\tclass\tbbox\tx0\ty0\tw\th\tarea\n")
    #                     fl.write(annot)

                # write classification images
                fldr_class_out = os.path.join('./synth_classification', Folder)
                make_folder(fldr_class_out)
                im_out = arr_2_pil(im)
                im_out.save(os.path.join(fldr_class_out, f"synthClass_{count:04}_{i:04}.jpg"))


    #         fldr_detc_out = os.path.join('./synth_detection', img_file_name)
    #         im_out = arr_2_pil(background)
    #         im_out.save(fldr_detc_out)


            if count % 50 == 0:
                time_elapsed = time.time() - start_time
                print(f"Count: {count}, Time: {round(time_elapsed/3600, 3)}")

            count+=1

Count: 0, Time: 0.001
Count: 50, Time: 0.041
Count: 100, Time: 0.082
Count: 150, Time: 0.122
Count: 200, Time: 0.162
Count: 250, Time: 0.202
Count: 300, Time: 0.241


In [ ]:
count

In [54]:
# %%time



# background_original = Image.open(background_source[8])
# background_original = np.array(background_original) / 255.0
# YY, XX, _ = background.shape


# background = background_original.copy()


# paths = sign_source[:3]

# num_images = len(paths)

# synth_images=[]
# max_w=0
# max_h=0

# for p in paths:
#     im, mask = get_augmented_image_mask(p, sizes=sizes)
#     H, W, _ = im.shape
#     if W > max_w:
#         max_w=W
#     if H > max_h:
#         max_h=H

#     synth_images.append({'image': im, 'mask': mask, 'shape': (H, W)})
    
# offset = (XX-max_w)//num_images

# for i in range(num_images):
#     synth_images[i]['range'] = (i*offset, i*offset+offset, YY-max_h)

# for i, sample in enumerate(synth_images):

#     xmin, xmax, ymax = sample['range']
#     im, mask = sample['image'], sample['mask']
#     yy, xx = sample['shape']

#     border=80
#     py = np.random.randint(0, ymax-yy-1-border)+border
#     px = np.random.randint(xmin, xmax-xx-1)

#     sign_overlay = background[py:py+yy, px:px+xx].copy()
#     im[mask < 1] = sign_overlay[mask < 1]

#     gamma = np.random.uniform(low=-0.1, high=0.1)
#     im = cv2.addWeighted(sign_overlay, 0.12, im, 0.8, gamma)
#     im[mask < 1] = sign_overlay[mask < 1]
#     im[im<0]=0
#     im[im>1]=1
    
#     synth_images[i]['image']=im

#     background[py:py+yy, px:px+xx]=im

# plt.figure(figsize=(15, 15))
# plt.imshow(background)
# plt.show()

# fig, ax = plt.subplots(ncols=num_images, figsize=(15, 15))
# for i, s in enumerate(synth_images):
#     ax[i].imshow(s['image'])
#     for a in ax:
#         a.axis('off')
# plt.show()

In [198]:
arr_2_pil(sample['image'])

In [ ]:
img_file_name = f

In [68]:
img_file_name = f"synth_{count:04}.jpg"
img_file_name

'synth_0000.jpg'

In [1137]:
annot = f"{img_file_name}\t{Folder}\t{x},{y},{w},{h}\t{x}\t{y}\t{w}\t{h}\t{w * h}\n"

In [ ]:
############### INPUTS ######################
number_of_generated_images = 100
Folder = 'test'
I = io.imread('w13-2.jpg')/255.0
##############################################

In [ ]:
f = 0
while f<number_of_generated_images:

    # Cut out the foreground layer
    foreground = I.copy()
    foreground[np.sum(I, axis=2) >= 2.9] = 0  # Setting surrounding pixels to zero (figure sth out if the sign itself is WHITE too)!!!

    # Augmentation of foregroundAug
    foreground_new = foregroundAug(foreground)

    ############ add filters (gaussian, ...)
    P_noise = np.random.randint(0, 100)
    if (P_noise >= 75):
        foreground_new = noisy("gauss", foreground_new)
    if (P_noise <= 10):
        foreground_new = noisy("speckle", foreground_new)
    if (10 < P_noise <= 20):
        foreground_new = noisy("s&p", foreground_new)


    # background selection
    background_fileName = np.random.choice(os.listdir("background/"))
    background = io.imread('./background/' + background_fileName) / 255.0

    ################################ Placement and size of the traffic signs on background #############################
    YY, XX = background.shape[:2]
    yy, xx = foreground_new.shape[:2]
    C = 0.05 + np.random.random() * 0.3
    R = YY * C / yy

    dy = int(foreground_new.shape[0] * R)
    dx = int(foreground_new.shape[1] * R)

    foreground_new = transform.resize(foreground_new, (dy, dx))
    # Py is the same as y - Px is the same as x
    Py = int(np.random.random() * background.shape[0])
    # if Py<dy/2:
    #     Py = int(dy/2+10)
    if Py > (background.shape[0] - dy):
        Py = int(background.shape[0] - dy - 10)

    Px = int(np.random.random() * background.shape[1])
    # if Px<dx/2:
    #     Px = int(dx/2+10)
    if Px > (background.shape[1] - dx):
        Px = int(background.shape[1] - dx - 10)

    y = Py # for labeling
    x = Px # for labeling
    w = dx # for labeling
    h = dy # for labeling
    ####################################################################################################################

    ############################ Overlaying the foreground on background ###############################################
    overlay = cv2.addWeighted(background[Py:Py + dy, Px:Px + dx], 0.5, foreground_new, 0.6, 0.2)
    weighted = background.copy()
    weighted[Py:Py + dy, Px:Px + dx] = overlay
    ####################################################################################################################

    ############################# Labeling ############################################################################
    Folder = Folder.lower()
    if os.path.isdir(Folder):
        pass
    else:
        os.mkdir(Folder)
    f += 1
    img_file_name = "{}".format(f) + ".jpg"
    cv2.imwrite(Folder + "/" + img_file_name[:-4] + "-{}".format(f) + ".jpg", cv2.cvtColor((255*weighted[Py:Py + dy, Px:Px + dx]).astype('float32'), cv2.COLOR_RGB2BGR)) # for classification
    cv2.imwrite("ALL_IMAGES" + "/" + img_file_name, cv2.cvtColor((255*weighted).astype('float32'), cv2.COLOR_RGB2BGR)) # for object detection

    annot = f"{img_file_name}\t{Folder}\t{x},{y},{w},{h}\t{x}\t{y}\t{w}\t{h}\t{w * h}\n"
    if os.path.isfile("Detection_labels_Syn.txt"):
        with open("Detection_labels_Syn.txt", "a") as fl:
            fl.write(annot)
    else:
        with open("Detection_labels_Syn.txt", "a") as fl:
            fl.write("filename\tclass\tbbox\tx0\ty0\tw\th\tarea\n")
            fl.write(annot)